In [1]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

# Evaluation metric

In [2]:
def smape(y_true, y_pred):
    '''
    https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error
    '''
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = (np.abs(y_true) + np.abs(y_pred)) / 2
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100*np.mean(smap)

# Prepare train and test sets

In [3]:
# based on https://www.kaggle.com/code/vadimkamaev/score-1-382/notebook

data_dir = '../data/'

def get_train_test():    
    # combine the original training set with the revealed data
    train = pd.concat([
        pd.read_csv(f'{data_dir}/train.csv'), 
        pd.read_csv(f'{data_dir}/revealed_test.csv')
    ]).sort_values(by=['cfips', 'first_day_of_month']).reset_index(0, drop=True)
    train['is_test'] = 0

    # drop the test data that has already been revealed
    test = pd.read_csv(f'{data_dir}/test.csv')
    drop_index = (test.first_day_of_month == '2022-11-01') | (test.first_day_of_month == '2022-12-01')
    test = test.loc[~drop_index, :]
    test['is_test'] = 1
    
    sub = pd.read_csv(f'{data_dir}/sample_submission.csv')
    return train, test, sub

train, test, sub = get_train_test()
print(len(train), len(test), len(sub))

128535 18810 25080


# Combine raw data

In [4]:
def get_raw(train, test):
    raw = pd.concat([train, test]).sort_values(['cfips', 'row_id']).reset_index(0, drop=True)
    raw['first_day_of_month'] = pd.to_datetime(raw["first_day_of_month"])
    raw['county'] = raw.groupby('cfips')['county'].ffill()
    raw['state'] = raw.groupby('cfips')['state'].ffill()
    raw['county_i'] = (raw['county'] + raw['state']).factorize()[0]
    raw['state_i'] = raw['state'].factorize()[0]
    raw['month_number'] = raw.groupby(['cfips'])['row_id'].cumcount()
    raw['y'] = raw['microbusiness_density']
    raw = raw.drop('microbusiness_density', axis=1)
    features = ['state_i']    
    return raw, features

raw, features = get_raw(train, test)


In [5]:
raw

,row_id,cfips,county,state,first_day_of_month,active,is_test,county_i,state_i,month_number,y
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,1249.0,0,0,0,0,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,1198.0,0,0,0,1,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,1269.0,0,0,0,2,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,1243.0,0,0,0,3,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,1243.0,0,0,0,4,2.993233
...,...,...,...,...,...,...,...,...,...,...,...
147340,56045_2023-02-01,56045,Weston County,Wyoming,2023-02-01,NaN,1,3134,50,42,NaN
147341,56045_2023-03-01,56045,Weston County,Wyoming,2023-03-01,NaN,1,3134,50,43,NaN
147342,56045_2023-04-01,56045,Weston County,Wyoming,2023-04-01,NaN,1,3134,50,44,NaN
147343,56045_2023-05-01,56045,Weston County,Wyoming,2023-05-01,NaN,1,3134,50,45,NaN


In [6]:
# os.environ['CUDA_VISIBLE_DEVICES']='0'
# raw['scale'] = (raw['first_day_of_month'] - raw['first_day_of_month'].min()).dt.days
# raw['scale'] = raw['scale'].factorize()[0]
# raw
# raw.groupby('cfips')['microbusiness_density'].shift(-1)/raw['microbusiness_density'] - 1
# raw['lastactive'] = raw.groupby('cfips')['active'].transform('last')


# Feature engineering: lag and rolling

In [7]:
def add_lag(raw, features, target='y', max_lag=8):
    for lag in range(1, max_lag):
        raw[f'{target}_lag_{lag}'] = raw.groupby('cfips')[target].shift(lag)
        features.append(f'{target}_lag_{lag}')        
        # raw[f'active_lag_{lag}'] = raw.groupby('cfips')['active'].diff(lag)        
        # feats.append(f'active_lag_{lag}')        
    return raw, features

def add_rolling(raw, features, target='y', lags=[1]):
    
    def get_rolling(s, window):
        return s.rolling(window, min_periods=1).sum()
        
    for lag in lags:
        for window in [2, 4, 6, 8, 10]:
            raw[f'{target}_roll_{window}_{lag}'] = raw.groupby('cfips')[f'{target}_lag_{lag}'].transform(get_rolling, window=window)
            features.append(f'{target}_roll_{window}_{lag}')
    return raw, features

# Feature engineering: internal and external census

In [8]:
def add_internal_census(raw, features):
    census = pd.read_csv(f'{data_dir}/census_starter.csv')
    census_cols = list(census.columns)
    census_cols.remove('cfips')
    raw = raw.merge(census, on='cfips', how='left')
    features += census_columns
    return raw, features

def add_external_census(raw, features):
    '''
    data: https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/counties/totals/co-est2021-alldata.csv
    schema: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2021/CO-EST2021-ALLDATA.pdf
    '''
    census = pd.read_csv(f'{data_dir}/co-est2021-alldata.csv', encoding='latin-1')
    census['cfips'] = census.STATE*1000 + census.COUNTY
    features += [
        'SUMLEV',
        'REGION',
        'DIVISION',
        'ESTIMATESBASE2020',
        'POPESTIMATE2020',
        'POPESTIMATE2021',
        'NPOPCHG2020',
        'NPOPCHG2021',
        'BIRTHS2020',
        'BIRTHS2021',
        'DEATHS2020',
        'DEATHS2021',
        'NATURALCHG2020',
        'NATURALCHG2021',
        'INTERNATIONALMIG2020',
        'INTERNATIONALMIG2021',
        'DOMESTICMIG2020',
        'DOMESTICMIG2021',
        'NETMIG2020',
        'NETMIG2021',
        'RESIDUAL2020',
        'RESIDUAL2021',
        'GQESTIMATESBASE2020',
        'GQESTIMATES2020',
        'GQESTIMATES2021',
        'RBIRTH2021',
        'RDEATH2021',
        'RNATURALCHG2021',
        'RINTERNATIONALMIG2021',
        'RDOMESTICMIG2021',
        'RNETMIG2021'
    ]
    raw = raw.merge(census, on='cfips', how='left')
    return raw, features

# Feature engineering: coordinates

In [9]:
def add_coords(raw, features):
    '''
    https://www.kaggle.com/datasets/alejopaullier/usa-counties-coordinates
    '''
    coords = pd.read_csv(f'{data_dir}/cfips_location.csv').drop('name', axis=1) 
    raw = raw.merge(coords, on='cfips')
    features += ['lng', 'lat']
    return raw, features

# Feature engineering: all

In [10]:
raw['target'] = raw.groupby('cfips')['y'].shift(-1)
raw['target'] = raw['target']/raw['y'] - 1
raw.loc[raw['cfips']==28055, 'target'] = 0.0
raw.loc[raw['cfips']==48269, 'target'] = 0.0
raw['lastactive'] = raw.groupby('cfips')['active'].transform('last')
raw, features = add_lag(raw, features, target='target')
raw, features = add_rolling(raw, features, target='target')

In [11]:
raw

,row_id,cfips,county,state,first_day_of_month,active,is_test,county_i,state_i,month_number,...,target_lag_3,target_lag_4,target_lag_5,target_lag_6,target_lag_7,target_roll_2_1,target_roll_4_1,target_roll_6_1,target_roll_8_1,target_roll_10_1
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,1249.0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,1198.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,-0.040833,-0.040833,-0.040833,-0.040833,-0.040833
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,1269.0,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,0.018433,0.018433,0.018433,0.018433,0.018433
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,1243.0,0,0,0,3,...,-0.040833,NaN,NaN,NaN,NaN,0.038777,-0.002056,-0.002056,-0.002056,-0.002056
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,1243.0,0,0,0,4,...,0.059265,-0.040833,NaN,NaN,NaN,-0.020489,-0.002056,-0.002056,-0.002056,-0.002056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147340,56045_2023-02-01,56045,Weston County,Wyoming,2023-02-01,NaN,1,3134,50,42,...,0.010000,0.000000,0.00,0.00,-0.009901,NaN,0.010000,0.010000,0.000099,0.020301
147341,56045_2023-03-01,56045,Weston County,Wyoming,2023-03-01,NaN,1,3134,50,43,...,NaN,0.010000,0.00,0.00,0.000000,NaN,0.010000,0.010000,0.000099,0.000099
147342,56045_2023-04-01,56045,Weston County,Wyoming,2023-04-01,NaN,1,3134,50,44,...,NaN,NaN,0.01,0.00,0.000000,NaN,NaN,0.010000,0.010000,0.000099
147343,56045_2023-05-01,56045,Weston County,Wyoming,2023-05-01,NaN,1,3134,50,45,...,NaN,NaN,NaN,0.01,0.000000,NaN,NaN,0.010000,0.010000,0.000099


# Model

In [12]:
def get_model():
    from sklearn.ensemble import VotingRegressor
    import lightgbm as lgb
    import xgboost as xgb
    import catboost as cat
    from sklearn.pipeline import Pipeline
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.impute import KNNImputer    

    # we should decrease the num_iterations of catboost
    cat_model = cat.CatBoostRegressor(
        iterations=800,
        loss_function="MAPE",
        verbose=0,
        grow_policy='SymmetricTree',
        learning_rate=0.035,
        colsample_bylevel=0.8,
        max_depth=5,
        l2_leaf_reg=0.2,
        # max_leaves = 17,
        subsample=0.70,
        max_bin=4096,
    )
    return cat_model

# Training

In [13]:
# ACT_THR = 140
# MONTH_1 = 39
# MONTH_last = 41
# raw['ypred_last'] = np.nan
# raw['ypred'] = np.nan
# raw['k'] = 1.
# raw['y'].fillna(0, inplace = True)

# for TS in range(MONTH_1, MONTH_last): #40):
#     print(TS)
   
#     model = get_model()
            
#     train_indices = (raw.is_test==0) & (raw.month_number  < TS) & (raw.month_number >= 1) & (raw.lastactive>ACT_THR) 
#     valid_indices = (raw.is_test==0) & (raw.month_number == TS) 
#     model.fit(
#         raw.loc[train_indices, features],
#         raw.loc[train_indices, 'target'].clip(-0.0043, 0.0045),

#     )

#     ypred = model.predict(raw.loc[valid_indices, features])
#     raw.loc[valid_indices, 'k'] = ypred + 1
#     raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'y']

#     # Validate
#     lastval = raw.loc[raw.month_number==TS, ['cfips', 'y']].set_index('cfips').to_dict()['y']
#     dt = raw.loc[raw.month_number==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
#     df = raw.loc[raw.month_number==(TS+1), 
#                  ['cfips', 'y', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
#     df['pred'] = df['cfips'].map(dt)
#     df['lastval'] = df['cfips'].map(lastval)
    
# #     df.loc[df['lastval'].isnull(), 'lastval'] = df.loc[df['lastval'].isnull(), 'microbusiness_density']    
    
#     df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
        
#     raw.loc[raw.month_number==(TS+1), 'ypred'] = df['pred'].values
#     raw.loc[raw.month_number==(TS+1), 'ypred_last'] = df['lastval'].values
    
#     print(f'TS: {TS}')
#     print('Last Value SMAPE:', smape(df['y'], df['lastval']) )
#     print('SMAPE:', smape(df['y'], df['pred']))
#     print()


# ind = (raw.month_number > MONTH_1)&(raw.month_number <= MONTH_last)

# print( 'SMAPE:', smape( raw.loc[ind, 'y'],  raw.loc[ind, 'ypred'] ) )
# print( 'Last Value SMAPE:', smape( raw.loc[ind, 'y'],  raw.loc[ind, 'ypred_last'] ) )

In [16]:
ACT_THR = 140
MONTH_1 = 39
MONTH_last = 41

raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
# raw['k'] = 1.
raw['y'].fillna(0, inplace = True)

for TS in range(MONTH_1, MONTH_last):
    print(TS)   
    model = get_model()
    train_idxs = (raw.is_test==0) & (raw.month_number  < TS) & (raw.month_number >= 1) & (raw.lastactive>ACT_THR) 
    valid_idxs = (raw.is_test==0) & (raw.month_number == TS) 
    print(raw.loc[train_idxs, 'y'])
    model.fit(
        raw.loc[train_idxs, features],
        raw.loc[train_idxs, 'target'].clip(-0.0043, 0.0045),
#         raw.loc[train_idxs, 'y'],
    )

    ypred = model.predict(raw.loc[valid_idxs, features])
    raw.loc[valid_idxs, 'k'] = (ypred + 1)
    raw.loc[valid_idxs, 'k'] = raw.loc[valid_idxs, 'k']*raw.loc[valid_idxs,'y']
#     raw.loc[valid_idxs, 'yhat'] = ypred
    
    # Validate
    ylast_dict = raw.loc[raw.month_number==TS, ['cfips', 'y']].set_index('cfips').to_dict()['y']
    print(len(ylast_dict))
#     yhat_dict = raw.loc[raw.month_number==TS, ['cfips', 'yhat']].set_index('cfips').to_dict()['yhat']
    yhat_dict = raw.loc[raw.month_number==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    df = raw.loc[raw.month_number == (TS+1), 
                 ['cfips', 'y', 
#                   'state', 
                  'lastactive', 
#                   'y_lag_1'
                 ]].reset_index(drop=True)
    df['yhat'] = df['cfips'].map(yhat_dict)
    df['ylast'] = df['cfips'].map(ylast_dict)
#     df.loc[df['lastval'].isnull(), 'lastval'] = df.loc[df['lastval'].isnull(), 'microbusiness_density']    
    
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'ylast']
        
    raw.loc[raw.month_number==(TS+1), 'ypred'] = df['yhat'].values
    raw.loc[raw.month_number==(TS+1), 'ypred_last'] = df['ylast'].values
    
    print(f'TS: {TS}')
    print('Last Value SMAPE:', smape(df['y'], df['ylast']) )
    print('SMAPE:', smape(df['y'], df['yhat']))

ind = (raw.month_number > MONTH_1)&(raw.month_number <= MONTH_last)

print( 'SMAPE:', smape( raw.loc[ind, 'y'],  raw.loc[ind, 'ypred'] ) )
print( 'Last Value SMAPE:', smape( raw.loc[ind, 'y'],  raw.loc[ind, 'ypred_last'] ) )

39
1         2.884870
2         3.055843
3         2.993233
4         2.993233
5         2.969090
            ...   
147285    3.126551
147286    3.225807
147287    3.209264
147288    3.209264
147289    3.126551
Name: y, Length: 91010, dtype: float64
3135
TS: 39
Last Value SMAPE: 1.889206717018118
SMAPE: 1.8973797476217276
40
1         2.884870
2         3.055843
3         2.993233
4         2.993233
5         2.969090
            ...   
147286    3.225807
147287    3.209264
147288    3.209264
147289    3.126551
147290    3.143093
Name: y, Length: 93405, dtype: float64
3135
TS: 40
Last Value SMAPE: 200.0
SMAPE: 200.0
SMAPE: 100.94868987381088
Last Value SMAPE: 100.94460335850906


In [17]:
raw.drop(['y_lag_2', 'y_lag_3', 'y_lag_4', 'y_lag_5'], axis=1)

KeyError: "['y_lag_2', 'y_lag_3', 'y_lag_4', 'y_lag_5'] not found in axis"